---
## First Cut Approach
---

**Objective of this step to predict the failure of machines' components (comp1, comp2 etc.) based on available data withoutperforming feature engineering and hyper-parameter tuning of model.**


**Steps to be followed:**

    1.Load all the data sets
    2.Perform one hot encoding for categorical attributes
    3.Merger all the data sets after step no-2.
    4.Split the data sets into train, cross validation and test set.
    5.Apply Xgboost classifier and random forest classifier on train data set.
    6.As the dataset is highly imbalance, so F1 score, Precision, Recall and Confusion matrix will be computed to 
      validate theperformance of models.

In [1]:
import pandas as pd
import numpy as np

## **Import DataSet**

In [2]:
telemetry = pd.read_csv("0_Dataset/PdM_telemetry.csv")
errors    = pd.read_csv("0_Dataset/PdM_errors.csv")
maint     = pd.read_csv("0_Dataset/PdM_maint.csv")
failures  = pd.read_csv("0_Dataset/PdM_failures.csv")
machines  = pd.read_csv("0_Dataset/PdM_machines.csv")

In [3]:
telemetry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876100 entries, 0 to 876099
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   datetime   876100 non-null  object 
 1   machineID  876100 non-null  int64  
 2   volt       876100 non-null  float64
 3   rotate     876100 non-null  float64
 4   pressure   876100 non-null  float64
 5   vibration  876100 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 40.1+ MB


In [4]:
errors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3919 entries, 0 to 3918
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   datetime   3919 non-null   object
 1   machineID  3919 non-null   int64 
 2   errorID    3919 non-null   object
dtypes: int64(1), object(2)
memory usage: 92.0+ KB


In [5]:
maint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3286 entries, 0 to 3285
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   datetime   3286 non-null   object
 1   machineID  3286 non-null   int64 
 2   comp       3286 non-null   object
dtypes: int64(1), object(2)
memory usage: 77.1+ KB


In [6]:
failures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   datetime   761 non-null    object
 1   machineID  761 non-null    int64 
 2   failure    761 non-null    object
dtypes: int64(1), object(2)
memory usage: 18.0+ KB


In [7]:
machines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   machineID  100 non-null    int64 
 1   model      100 non-null    object
 2   age        100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB


## Formatting Date Time Field

In [8]:
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'],format="%Y-%m-%d %H:%M:%S")
errors['datetime'] = pd.to_datetime(errors['datetime'],format="%Y-%m-%d %H:%M:%S")
maint['datetime'] = pd.to_datetime(maint['datetime'],format="%Y-%m-%d %H:%M:%S")
failures['datetime'] = pd.to_datetime(failures['datetime'],format="%Y-%m-%d %H:%M:%S")

In [9]:
telemetry.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876100 entries, 0 to 876099
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   datetime   876100 non-null  datetime64[ns]
 1   machineID  876100 non-null  int64         
 2   volt       876100 non-null  float64       
 3   rotate     876100 non-null  float64       
 4   pressure   876100 non-null  float64       
 5   vibration  876100 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 40.1 MB


### Telemetry Table

In [10]:
telemetry.head(3)

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847


### Fine Tune Error Table

In [11]:
errors.head(3)

,datetime,machineID,errorID
0,2015-01-03 07:00:00,1,error1
1,2015-01-03 20:00:00,1,error3
2,2015-01-04 06:00:00,1,error5


In [12]:
errors = pd.get_dummies(errors)
errors.head(3)

,datetime,machineID,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5
0,2015-01-03 07:00:00,1,1,0,0,0,0
1,2015-01-03 20:00:00,1,0,0,1,0,0
2,2015-01-04 06:00:00,1,0,0,0,0,1


### Fine Tune Maintainance Table

In [13]:
maint.head(3)

,datetime,machineID,comp
0,2014-06-01 06:00:00,1,comp2
1,2014-07-16 06:00:00,1,comp4
2,2014-07-31 06:00:00,1,comp3


In [14]:
maint = pd.get_dummies(maint)
maint.head(3)

,datetime,machineID,comp_comp1,comp_comp2,comp_comp3,comp_comp4
0,2014-06-01 06:00:00,1,0,1,0,0
1,2014-07-16 06:00:00,1,0,0,0,1
2,2014-07-31 06:00:00,1,0,0,1,0


### Fine Tune Machines Table

In [15]:
machines.head(3)

,machineID,model,age
0,1,model3,18
1,2,model4,7
2,3,model3,8


In [16]:
machines = pd.get_dummies(machines)
machines.head(3)

,machineID,age,model_model1,model_model2,model_model3,model_model4
0,1,18,0,0,1,0
1,2,7,0,0,0,1
2,3,8,0,0,1,0


### Failure Table

In [17]:
failures.head(3)

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2


## Merging of Telemetry and Error Table

In [18]:
telemetry.head(3)

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847


In [19]:
errors.head(3)

,datetime,machineID,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5
0,2015-01-03 07:00:00,1,1,0,0,0,0
1,2015-01-03 20:00:00,1,0,0,1,0,0
2,2015-01-04 06:00:00,1,0,0,0,0,1


In [21]:
tel_error_merge = telemetry[['datetime', 'machineID']].merge(errors, on = ['datetime', 'machineID'], how = 'outer').fillna(0)

In [22]:
tel_error_merge.head(3)

,datetime,machineID,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5
0,2015-01-01 06:00:00,1,0.0,0.0,0.0,0.0,0.0
1,2015-01-01 07:00:00,1,0.0,0.0,0.0,0.0,0.0
2,2015-01-01 08:00:00,1,0.0,0.0,0.0,0.0,0.0


In [23]:
telemetry.shape

(876100, 6)

In [24]:
errors.shape

(3919, 7)

In [25]:
tel_error_merge.shape

(876403, 7)

## Merging of Telemetry & Maintanance Table

In [26]:
telemetry.head(2)

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973


In [27]:
maint.head(2)

,datetime,machineID,comp_comp1,comp_comp2,comp_comp3,comp_comp4
0,2014-06-01 06:00:00,1,0,1,0,0
1,2014-07-16 06:00:00,1,0,0,0,1


In [28]:
tel_maint_merge = telemetry[['datetime', 'machineID']].merge(maint, on = ['datetime', 'machineID'], 
                                                             how = 'outer').fillna(0)

In [29]:
tel_maint_merge.head(2)

,datetime,machineID,comp_comp1,comp_comp2,comp_comp3,comp_comp4
0,2015-01-01 06:00:00,1,0.0,0.0,0.0,0.0
1,2015-01-01 07:00:00,1,0.0,0.0,0.0,0.0


In [30]:
telemetry.shape

(876100, 6)

In [31]:
tel_maint_merge.shape

(877223, 6)

## Merging of Telemetry, Maintanance & Machine Table

In [32]:
tel_maint_merge.head(2)

,datetime,machineID,comp_comp1,comp_comp2,comp_comp3,comp_comp4
0,2015-01-01 06:00:00,1,0.0,0.0,0.0,0.0
1,2015-01-01 07:00:00,1,0.0,0.0,0.0,0.0


In [33]:
machines.head(2)

,machineID,age,model_model1,model_model2,model_model3,model_model4
0,1,18,0,0,1,0
1,2,7,0,0,0,1


In [34]:
tel_maint_mac_merge = tel_maint_merge.merge(machines, on = 'machineID', how = 'left').fillna(0)

In [35]:
tel_maint_mac_merge.head(2)

,datetime,machineID,comp_comp1,comp_comp2,comp_comp3,comp_comp4,age,model_model1,model_model2,model_model3,model_model4
0,2015-01-01 06:00:00,1,0.0,0.0,0.0,0.0,18,0,0,1,0
1,2015-01-01 07:00:00,1,0.0,0.0,0.0,0.0,18,0,0,1,0


In [36]:
tel_maint_mac_merge.shape

(877223, 11)

## Merging of Telemetry and Failure Table

In [37]:
telemetry.head(2)

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973


In [38]:
failures.head(2)

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1


In [39]:
tel_fail_merge = telemetry[['datetime', 'machineID']].merge(failures, on = ['datetime', 'machineID'],
                                                           how = 'left')

In [40]:
tel_fail_merge.head(2)

,datetime,machineID,failure
0,2015-01-01 06:00:00,1,NaN
1,2015-01-01 07:00:00,1,NaN


In [41]:
tel_fail_merge.shape

(876142, 3)

## Final Data Frame Preparation

In [42]:
final_df = telemetry.merge(tel_error_merge, on=['datetime', 'machineID'], how = 'left')
final_df.head(2)

,datetime,machineID,volt,rotate,pressure,vibration,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,0.0,0.0,0.0,0.0,0.0
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,0.0,0.0,0.0,0.0,0.0


In [43]:
final_df = final_df.merge(tel_maint_mac_merge, on=['datetime', 'machineID'], how = 'left')
final_df.head(2)

,datetime,machineID,volt,rotate,pressure,vibration,errorID_error1,errorID_error2,errorID_error3,errorID_error4,errorID_error5,comp_comp1,comp_comp2,comp_comp3,comp_comp4,age,model_model1,model_model2,model_model3,model_model4
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,0,0,1,0
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,0,0,1,0


In [44]:
final_df = final_df.merge(tel_fail_merge, on=['datetime', 'machineID'], how = 'left')
final_df.head()

,datetime,machineID,volt,rotate,pressure,vibration,errorID_error1,errorID_error2,errorID_error3,errorID_error4,...,comp_comp1,comp_comp2,comp_comp3,comp_comp4,age,model_model1,model_model2,model_model3,model_model4,failure
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18,0,0,1,0,NaN
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18,0,0,1,0,NaN
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18,0,0,1,0,NaN
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18,0,0,1,0,NaN
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18,0,0,1,0,NaN


In [45]:
final_df.shape

(877209, 21)

In [46]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 877209 entries, 0 to 877208
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   datetime        877209 non-null  datetime64[ns]
 1   machineID       877209 non-null  int64         
 2   volt            877209 non-null  float64       
 3   rotate          877209 non-null  float64       
 4   pressure        877209 non-null  float64       
 5   vibration       877209 non-null  float64       
 6   errorID_error1  877209 non-null  float64       
 7   errorID_error2  877209 non-null  float64       
 8   errorID_error3  877209 non-null  float64       
 9   errorID_error4  877209 non-null  float64       
 10  errorID_error5  877209 non-null  float64       
 11  comp_comp1      877209 non-null  float64       
 12  comp_comp2      877209 non-null  float64       
 13  comp_comp3      877209 non-null  float64       
 14  comp_comp4      877209 non-null  flo

## EDA on Final DataFrame

In [47]:
final_df['failure'].info()

<class 'pandas.core.series.Series'>
Int64Index: 877209 entries, 0 to 877208
Series name: failure
Non-Null Count  Dtype 
--------------  ----- 
1122 non-null   object
dtypes: object(1)
memory usage: 13.4+ MB


In [48]:
final_df['failure'] = final_df['failure'].astype('str')

In [49]:
final_df['failure'].info()

<class 'pandas.core.series.Series'>
Int64Index: 877209 entries, 0 to 877208
Series name: failure
Non-Null Count   Dtype 
--------------   ----- 
877209 non-null  object
dtypes: object(1)
memory usage: 13.4+ MB


## Replacing 'nan' with 'none'

In [50]:
final_df['failure'].unique()

array(['nan', 'comp4', 'comp1', 'comp2', 'comp3'], dtype=object)

In [51]:
final_df.replace(to_replace='nan',
                 value='none',
                 inplace = True)

In [52]:
final_df['failure'].unique()

array(['none', 'comp4', 'comp1', 'comp2', 'comp3'], dtype=object)

## Save the Final DataFrame

In [53]:
# final_df.to_csv('1_Output/final_df_first_cut.csv')